# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up libraries

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


# Functions

Calculate RSS

In [2]:
def get_residual_sum_of_squares(input_feature, output, intercept,slope):
    RSS_R = (output - np.dot(input_feature, slope) - intercept)
    RSS_L = np.transpose(RSS_R)
    RSS = np.dot(RSS_L,RSS_R)
    return(RSS)

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [3]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, \
'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, \
'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, \
'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, \
'id':str, 'sqft_lot':int, 'view':int}

sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [4]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
from sklearn import linear_model  # using scikit-learn

all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']



Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = linear_model.Lasso(alpha=5e2, normalize=True) # set parameters
model_all.fit(sales[all_features], sales['price']) # learn weights

Lasso(alpha=500.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Find what features had non-zero weight.

In [17]:
print(model_all.coef_)
print(all_features)


[     0.              0.              0.            134.43931396      0.
      0.              0.              0.              0.              0.
  24750.00458561      0.          61749.10309071      0.              0.
     -0.              0.        ]
['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets. Download the provided csv files containing training, validation and test sets.

In [8]:
testing = pd.read_csv('wk3_kc_house_test_data.csv', dtype=dtype_dict)
training = pd.read_csv('wk3_kc_house_train_data.csv', dtype=dtype_dict)
validation = pd.read_csv('wk3_kc_house_valid_data.csv', dtype=dtype_dict)

Make sure to create the 4 features as we did in #1:

In [9]:
testing['sqft_living_sqrt'] = testing['sqft_living'].apply(sqrt)
testing['sqft_lot_sqrt'] = testing['sqft_lot'].apply(sqrt)
testing['bedrooms_square'] = testing['bedrooms']*testing['bedrooms']
testing['floors'] = testing['floors'].astype(float) 
testing['floors_square'] = testing['floors']*testing['floors']

training['sqft_living_sqrt'] = training['sqft_living'].apply(sqrt)
training['sqft_lot_sqrt'] = training['sqft_lot'].apply(sqrt)
training['bedrooms_square'] = training['bedrooms']*training['bedrooms']
training['floors'] = training['floors'].astype(float) 
training['floors_square'] = training['floors']*training['floors']

validation['sqft_living_sqrt'] = validation['sqft_living'].apply(sqrt)
validation['sqft_lot_sqrt'] = validation['sqft_lot'].apply(sqrt)
validation['bedrooms_square'] = validation['bedrooms']*validation['bedrooms']
validation['floors'] = validation['floors'].astype(float) 
validation['floors_square'] = validation['floors']*validation['floors']

Now for each l1_penalty in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type np.logspace(1, 7, num=13).)

In [10]:
l1_penalty = np.logspace(1, 7, num=13)

In [11]:
for i in range(len(l1_penalty)):
    model = linear_model.Lasso(alpha=l1_penalty[i], normalize=True)
    model.fit(training[all_features], training['price']) 
    print(model.coef_)
    input_feature = np.array(validation[all_features])
    output = np.array(validation['price'])
    intercept = model.intercept_
    slope = model.coef_
    RSS = get_residual_sum_of_squares(input_feature, output, intercept,slope)
    print(l1_penalty[i])
    print(RSS)

[ -1.61445628e+04   3.73245384e+02   5.08412433e+04   6.17853560e+02
  -4.44113549e+04   7.85623065e-01  -7.01194765e+02  -0.00000000e+00
   5.01420046e+03   6.19488752e+05   3.80418557e+04   2.49987718e+04
   1.28716235e+05   0.00000000e+00   0.00000000e+00  -3.29383118e+03
   1.00573209e+01]
10.0
3.982133273e+14
[ -1.73819665e+04   1.19275717e+02   4.26517309e+04   4.19222928e+02
  -2.46029253e+04   2.39763524e-01  -3.55793616e+02   0.00000000e+00
   5.36190642e+03   5.98660591e+05   4.05138886e+04   1.74113669e+04
   1.25151941e+05   0.00000000e+00   0.00000000e+00  -3.33566856e+03
   4.30857032e+00]
31.6227766017
3.99041900253e+14
[ -1.00629243e+04   0.00000000e+00   2.21030560e+04   1.69564623e+02
  -0.00000000e+00  -0.00000000e+00  -9.88978280e+01   0.00000000e+00
   2.01034593e+03   5.48282396e+05   4.25386798e+04   2.85578129e+03
   1.19373956e+05   0.00000000e+00   0.00000000e+00  -3.03956199e+03
   0.00000000e+00]
100.0
4.29791604073e+14
[ -0.00000000e+00   0.00000000e+00   0

In [82]:
l1_penalty = range(0, 100)
for i in range(len(l1_penalty)):
    model = linear_model.Lasso(alpha=i, normalize=True)
    model.fit(training[all_features], training['price']) 
    print(model.coef_)
    input_feature = np.array(validation[all_features])
    output = np.array(validation['price'])
    intercept = model.intercept_
    slope = model.coef_
    RSS = get_residual_sum_of_squares(input_feature, output, intercept,slope)
    print(i)
    print(RSS)

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:454: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  positive)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


[ -1.56087369e+04   4.90449118e+02   5.55241805e+04   3.01881776e+02
  -5.27234923e+04   1.04698302e+00  -8.69393423e+02  -7.40621830e+04
   2.58665433e+04   6.30120166e+05   3.65335655e+04   2.77905061e+04
   1.30267287e+05   4.02311722e+02   3.98296090e+02  -3.29970277e+03
   1.27218054e+01]
0
4.03534542181e+14
[ -1.56236177e+04   4.77769244e+02   5.50587299e+04   3.37919598e+02
  -5.19625683e+04   1.01946113e+00  -8.51214598e+02  -5.99596685e+04
   2.18882591e+04   6.28981105e+05   3.67122488e+04   2.75677291e+04
   1.30137304e+05   3.57897243e+02   3.54800343e+02  -3.29665740e+03
   1.24553480e+01]
1
4.0281484261e+14
[ -1.56384984e+04   4.65089371e+02   5.45932794e+04   3.73957421e+02
  -5.12016443e+04   9.91939245e-01  -8.33035773e+02  -4.58571539e+04
   1.79099749e+04   6.27842043e+05   3.68909321e+04   2.73449521e+04
   1.30007321e+05   3.13482765e+02   3.11304597e+02  -3.29361202e+03
   1.21888905e+01]
2
4.02161414882e+14
[ -1.56533792e+04   4.52409497e+02   5.41278288e+04   4.

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [85]:
model = linear_model.Lasso(alpha=20, normalize=True)
model.fit(validation[all_features], validation['price']) 
input_feature = np.array(testing[all_features])
output = np.array(testing['price'])
intercept = model.intercept_
slope = model.coef_
RSS = get_residual_sum_of_squares(input_feature, output, intercept,slope)
RSS/1e14

1.012070111625281

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [86]:
np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)

15

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [20]:
l1_penalty_values = np.logspace(1, 4, num=20)
l1_penalty_values

array([    10.        ,     14.38449888,     20.69138081,     29.76351442,
           42.81332399,     61.58482111,     88.58667904,    127.42749857,
          183.29807108,    263.66508987,    379.26901907,    545.55947812,
          784.75997035,   1128.83789168,   1623.77673919,   2335.72146909,
         3359.81828628,   4832.93023857,   6951.92796178,  10000.        ])

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [72]:
import math
l1_penalty_values = np.logspace(1, 4, num=20)  #initialize the l1 penalty values
previous_max = 0
previous_min = 0
max_l1_pos = 0
min_l1_pos = 0
max_l1_penalty = l1_penalty_values[0] # 10
min_l1_penalty = l1_penalty_values[-1] # 10000
max_nonzeros = 7
Num = 20
#while abs(max_l1_penalty - previous_max) >= 0.01 or abs(min_l1_penalty - previous_min) >= 0.01 or flag != True:
while int(max_l1_penalty) - int(max_l1_pos) != 0 or int(min_l1_penalty) - int(min_l1_pos)!=0:
#    flag = True
    print('min and max:',min_l1_penalty,max_l1_penalty)
    print(math.log10(min_l1_penalty),math.log10(max_l1_penalty))
###################################################################################
# if max l1 penalty equal previous max penalty, then enlarge the num of values in the range
# if not, just update the range
    if max_l1_penalty == previous_max or min_l1_penalty == previous_min:
        Num = Num+20
        if max_l1_penalty > min_l1_penalty:
            l1_penalty_values = np.logspace(math.log10(min_l1_penalty), math.log10(max_l1_penalty), num=Num)
            
        else:
            l1_penalty_values = np.logspace(math.log10(max_l1_penalty), math.log10(min_l1_penalty), num=Num)
    else:
        if max_l1_penalty > min_l1_penalty:
            l1_penalty_values = np.logspace(math.log10(min_l1_penalty), math.log10(max_l1_penalty), num=Num)
        else:
            l1_penalty_values = np.logspace(math.log10(max_l1_penalty), math.log10(min_l1_penalty), num=Num)
    print(l1_penalty_values)
    previous_max = max_l1_penalty
    previous_min = min_l1_penalty
    max_l1_penalty = l1_penalty_values[0]
    min_l1_penalty = l1_penalty_values[-1]
    for i in range(len(l1_penalty_values)):
        
        model = linear_model.Lasso(alpha=l1_penalty_values[i], normalize=True)
        model.fit(training[all_features], training['price']) 
        nonzero_value = np.count_nonzero(model.coef_) + np.count_nonzero(model.intercept_)
#        if nonzero_value != 7:
#            flag = False
#            print(False)
        print(l1_penalty_values[i],nonzero_value)
############################################################################
# if nonzero_value = 7, then we need to find the lowest for min and highest value for max
        if nonzero_value == max_nonzeros and l1_penalty_values[i] <min_l1_penalty:
            min_l1_penalty = l1_penalty_values[i]
            min_pos = i
        if nonzero_value == max_nonzeros and l1_penalty_values[i] > max_l1_penalty:
            max_l1_penalty = l1_penalty_values[i]
            max_pos = i
    min_l1_penalty = l1_penalty_values[min_pos-1]  
    max_l1_penalty = l1_penalty_values[max_pos + 1]
    min_l1_pos = l1_penalty_values[min_pos]
    max_l1_pos = l1_penalty_values[max_pos]
print(min_l1_penalty, max_l1_penalty)

min and max: 10000.0 10.0
4.0 1.0
[    10.             14.38449888     20.69138081     29.76351442
     42.81332399     61.58482111     88.58667904    127.42749857
    183.29807108    263.66508987    379.26901907    545.55947812
    784.75997035   1128.83789168   1623.77673919   2335.72146909
   3359.81828628   4832.93023857   6951.92796178  10000.        ]
False
10.0 15
False
14.3844988829 15
False
20.6913808111 15
False
29.7635144163 15
False
42.8133239872 13
False
61.5848211066 12
False
88.586679041 11
False
127.42749857 10
183.298071083 7
False
263.665089873 6
False
379.269019073 6
False
545.559478117 6
False
784.759970351 5
False
1128.83789168 3
False
1623.77673919 3
False
2335.72146909 2
False
3359.81828628 1
False
4832.93023857 1
False
6951.92796178 1
False
10000.0 1
min and max: 127.42749857 263.665089873
2.1052631578947367 2.4210526315789473
[ 127.42749857  132.39868038  137.56379717  142.93041469  148.50639386
  154.29990226  160.31942612  166.57378271  173.07213328  179.8239

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [74]:
l1_penalty_min = 153
l1_penalty_max = 206

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [76]:
l1_penalty_values = range(l1_penalty_min,l1_penalty_max)
lowest_RSS = 5e14
lowest_RSS_pos = 1
for i in l1_penalty_values:
    model = linear_model.Lasso(alpha=i, normalize=True)
    model.fit(training[all_features], training['price'])
    input_feature = np.array(validation[all_features])
    output = np.array(validation['price'])
    intercept = model.intercept_
    slope = model.coef_
    RSS = get_residual_sum_of_squares(input_feature, output, intercept,slope)
    print(i)
    print(RSS)
    if RSS < lowest_RSS:
        lowest_RSS = RSS
        lowest_RSS_pos = i
print(lowest_RSS)
print(lowest_RSS_pos)

153
4.39710194789e+14
154
4.39830405618e+14
155
4.39927847512e+14
156
4.40026534452e+14
157
4.40128076553e+14
158
4.40228847334e+14
159
4.40330640518e+14
160
4.40433412978e+14
161
4.40537176018e+14
162
4.40641968738e+14
163
4.40747748743e+14
164
4.40854511824e+14
165
4.40962257981e+14
166
4.41070987216e+14
167
4.41180690797e+14
168
4.41291376292e+14
169
4.41403044727e+14
170
4.41515696112e+14
171
4.41629330448e+14
172
4.41743947735e+14
173
4.41859547972e+14
174
4.4197613116e+14
175
4.42093697299e+14
176
4.42212246388e+14
177
4.4233177819e+14
178
4.42452285744e+14
179
4.42573775258e+14
180
4.42696265862e+14
181
4.42819739537e+14
182
4.42944196281e+14
183
4.43069636096e+14
184
4.43196058981e+14
185
4.43323464937e+14
186
4.43451853962e+14
187
4.43581226058e+14
188
4.43711581224e+14
189
4.4384291946e+14
190
4.43977503726e+14
191
4.44110777875e+14
192
4.44245021724e+14
193
4.44380248203e+14
194
4.44516458318e+14
195
4.44653639198e+14
196
4.4479179886e+14
197
4.44930940799e+14
198
4.45071065

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [80]:
model = linear_model.Lasso(alpha=153, normalize=True) #  bathrooms, sqft_living, sqft_lot_sqrt, floors_square, waterfront, view, condition, 
# grade,sqrt_above, yr_built, 
model.fit(validation[all_features], validation['price'])
print(model.coef_)
print(all_features)

[ -0.00000000e+00  -0.00000000e+00   6.13439476e+03   1.42857834e+02
   0.00000000e+00  -0.00000000e+00  -0.00000000e+00   0.00000000e+00
   0.00000000e+00   4.51066315e+05   4.91455514e+04   0.00000000e+00
   1.18688360e+05   0.00000000e+00   0.00000000e+00  -2.32715371e+03
   1.77852582e+00]
['bedrooms', 'bedrooms_square', 'bathrooms', 'sqft_living', 'sqft_living_sqrt', 'sqft_lot', 'sqft_lot_sqrt', 'floors', 'floors_square', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']
